In [1]:
import numpy as np
import random as rd
from sklearn.tree import DecisionTreeRegressor

In [37]:
class env:
    def __init__(self):
        self.nS = 3
        self.nA = 3
        self.reward = np.matrix([[1,0,0],[0.5,0,0.5],[0.5,0.5,0]])
        self.P = {0: np.matrix([[0.8,0.1,0.1],[0.5,0,0.5],[0.2,0.4,0.4]]), 1: np.matrix([[0.3,0.3,0.4],[0.2,0.1,0.7],[0.4,0.4,0.2]]), 2: np.matrix([[0.1,0.5,0.4],[0.3,0.5,0.2],[0,0.1,0.9]])}
    
    def offline_set(self, N, k):
        #we use T_i = k in this setting
        offline_set = {}
        for i in range(N):
            result = []
            for j in range(k):
                if j == 0:
                    state = rd.randint(0,self.nS-1)
                else:
                    num = rd.random()
                    target = self.P[result[len(result)-3]][result[len(result)-2],:]
                    if num <= target[0,0]:
                        state = 0
                    elif num <= target[0,0]+target[0,1]:
                        state = 1
                    else:
                        state = 2
                action = rd.randint(0,self.nA-1)
                reward = self.reward[int(state), int(action)]
                result.append(int(state))
                result.append(int(action))
                result.append(reward)
            result.append(rd.randint(0,self.nS-1))
            offline_set[i] = result
        return offline_set
        

In [38]:
#set up the environment
env1 = env()
#create offline dataset
data = env1.offline_set(10,500)

In [17]:
def transformation(data, n, Q):
    """
    Transform the list into a dataset.
    
    """
    index = 0
    
    for j in range(len(data[n])//3):
        if j == 0:
            result = np.array([[data[n][index],data[n][index+1], Q[data[n][index],data[n][index+1]]]])
        else:
            result = np.row_stack((result, [data[n][index], data[n][index+1], Q[data[n][index],data[n][index+1]]]))
        index += 3
    return result
        

In [21]:
def Fitted_Q_iteration(data, nS, nA, N, theta=0.00001, discount_factor=0.5):
    """
    Find nearly-optimal policy via FQI, details can be found in chapter.
    
    Args:
        data: A dictionary, with key represent the index of trajectory, each trajectory is a list;
        nS: The number of states in the environment;
        nA: The number of actions in the environment;
        N: The number of trajectories collected;
        theta: We stop iteration once our Q function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        A tuple (policy, Q) of the nearly-optimal policy and the corresponding estimated Q function.
    """
    Q_1 = np.zeros([nS,nA])
    Q = []
    Q.append(Q_1)
    i = 0
    while True:
        index = 0
        Q_2 = np.zeros([nS,nA])
        for j in range(len(data[i])//3):
            Q_2[data[i][index],data[i][index+1]] = data[i][index+2] + discount_factor * Q[len(Q)-1][data[i][index+3],np.argmax(Q[len(Q)-1][data[i][index+3],:])]
            index += 3
        Q.append(Q_2)
        tree_data = transformation(data, i, Q[len(Q)-1])
        X = tree_data[:, 0:2]
        y = tree_data[:, 2]
        tree = DecisionTreeRegressor(random_state = 0)
        tree.fit(X, y)
        ##use tree-based method as an example, here only use one tree
        Q_2 = np.zeros([nS, nA])
        for t in range(nS):
            for l in range(nA):
                K = np.zeros([nS, nA])
                s = 0
                tree_index = 0
                for k in range(len(data[i])//3):
                    s += (tree.predict(np.array([[t,l]]))[0] == tree.predict(np.array([[data[i][tree_index],data[i][tree_index+1]]]))[0])
                    tree_index += 3
                for m in range(nS):
                    for n in range(nA):
                        K[m, n] = (tree.predict(np.array([[t,l]]))[0] == tree.predict(np.array([[m,n]]))[0])/s
                tree_index = 0
                for k in range(len(data[i])//3):
                    Q_2[t,l] += K[data[i][tree_index], data[i][tree_index+1]]*(data[i][tree_index+2]+discount_factor*Q[len(Q)-1][data[i][tree_index+3],np.argmax(Q[len(Q)-1][data[i][tree_index+3],:])])
                    tree_index += 3    
        Q.append(Q_2)
        i += 1
        if i == N-1:
            break
        elif np.max(Q[len(Q)-1]-Q[len(Q)-3]) < theta:
            break
    
    Q_f = Q[len(Q)-1]
    
    policy = np.zeros([nS, nA])
    for s in range(nS):
        best_action = np.argmax(Q_f[s,:])
        policy[s, best_action] = 1.0
    
    return policy, Q_f
    
    

In [39]:
opt_policy, Q = Fitted_Q_iteration(data = data, nS = env1.nS, nA = env1.nA, N = 100)

In [40]:
opt_policy

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [41]:
Q

array([[1.75365985, 0.69655029, 0.57571014],
       [1.14213765, 0.5830127 , 1.16816201],
       [1.06227229, 1.11340011, 0.55223939]])